In [3]:
!pip install -U langchain-ollama langchain-openai

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!ls /content/drive/MyDrive/review_dataset

w_review_train.csv  w_review_train.parquet


In [6]:
import duckdb

con = duckdb.connect(database=':memory:', read_only=False)

In [7]:
df = con.execute("SELECT * FROM read_csv_auto('/content/drive/MyDrive/review_dataset/w_review_train.csv', parallel=false, encoding='UTF-8', ignore_errors=true)").fetchdf()

df.head()

,column0,column1
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...,4
2,ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...,3
3,รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...,5
4,อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...,5


In [8]:
df.shape

(40000, 2)

In [9]:
con.execute("COPY df TO '/content/drive/MyDrive/review_dataset/w_review_train.parquet' (FORMAT PARQUET)")

In [10]:
parquet_avg_rating_direct = con.execute("SELECT AVG(column1) FROM '/content/drive/MyDrive/review_dataset/w_review_train.parquet' WHERE column0 LIKE '%อร่อย%'").fetchone()[0]
parquet_avg_rating_direct

3.8313027179006562

In [11]:
parquet_file_path = '/content/drive/MyDrive/review_dataset/w_review_train.parquet'

In [12]:
parquet_avg_rating_direct = con.execute(f"SELECT AVG(column1) FROM '{parquet_file_path}' WHERE column0 LIKE '%อร่อย%'").fetchone()[0]
parquet_avg_rating_direct

3.8313027179006562

In [13]:
coffee_keywords = [
    'กาแฟ', 'คาเฟ่', 'coffee', 'cafe', 'เอสเปรสโซ่', 'ลาเต้',
    'มอคค่า', 'คาปูชิโน่', 'เฟรนช์เพรส', 'cold brew', 'drip',
    'espresso', 'latte', 'americano', 'macchiato', 'brew',
    'specialty', 'slow bar', 'ร้านกาแฟ', 'ร้านคาเฟ่', 'กาแฟสด',
    'เมนูกาแฟ', 'กาแฟนม', 'กาแฟดำ', 'ร้านบาริสต้า', 'barista', 'coffee shop'
]

where_clause = " OR ".join([f"column0 LIKE '%{keyword}%'" for keyword in coffee_keywords])

In [14]:
%%time
coffee_reviews_df = con.execute(f"SELECT * FROM '{parquet_file_path}' WHERE {where_clause}").fetchdf()

CPU times: user 1.92 s, sys: 50.2 ms, total: 1.97 s
Wall time: 1.99 s


In [15]:
csv_file_path = '/content/drive/MyDrive/review_dataset/w_review_train.csv'

In [16]:
%%time
coffee_reviews_df_csv = con.execute(f"SELECT * FROM read_csv_auto('{csv_file_path}', parallel=false, encoding='UTF-8', ignore_errors=true) WHERE {where_clause}").fetchdf()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 2.13 s, sys: 61.1 ms, total: 2.19 s
Wall time: 2.23 s


In [17]:
coffee_reviews_df.head()

,column0,column1
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...,4
2,สารภาพว่าไม่เคยคิดจะไปต่อคิวซื้อมากินเองครับ บ...,3
3,ร้านอาหารญี่ปุ่นร้านนี้ ใจจริงไม่อยากแนะนำเลยค...,5
4,เดือนแรกที่เค้าต่อคิวกัน 2 - 3 ชั่วโมง เราก็ว่...,5


In [18]:
coffee_reviews_df.shape

(4981, 2)

In [19]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [20]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [21]:
!ollama pull gemma3:270m

In [22]:
!ollama list

NAME           ID              SIZE      MODIFIED               
gemma3:270m    e7d36fb2c3b3    291 MB    Less than a second ago    


In [23]:
!ollama run gemma3:270m "สวัสดี"

สวัสดีค่ะ 😊



In [24]:
!curl -s http://127.0.0.1:11434/api/tags

{"models":[{"name":"gemma3:270m","model":"gemma3:270m","modified_at":"2025-11-07T01:51:35.24634312Z","size":291554930,"digest":"e7d36fb2c3b3293cfe56d55889867a064b3a2b22e98335f2e6e8a387e081d6be","details":{"parent_model":"","format":"gguf","family":"gemma3","families":["gemma3"],"parameter_size":"268.10M","quantization_level":"Q8_0"}}]}

In [25]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [26]:
llm = OllamaLLM(model="gemma3:270m")

In [27]:
response = llm.invoke("สวัสดี")
print(response)

สวัสดีค่ะ ยินดีที่ได้รู้จักค่ะ 😊



In [28]:
template = """
คุณเป็นครูสอน Python
คำถาม : {question}
"""

In [29]:
prompt = PromptTemplate(
    input_variables=["question"],
    template=template
)

In [30]:
formatted_prompt = prompt.format(question="เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10")
print(formatted_prompt)


คุณเป็นครูสอน Python
คำถาม : เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10



In [31]:
chain = prompt | llm

In [32]:
response = chain.invoke({"question": "เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10"})

In [33]:
response

'แน่นอนค่ะ! ผมพร้อมที่จะช่วยคุณเขียนโค้ด Python เพื่อหาผลรวมของตัวเลข 1 ถึง 10\n\nเพื่อให้ผมสามารถช่วยคุณได้ดีที่สุด รบกวนข้อมูลที่คุณมีหน่อยนะคะ\n\n*   **ข้อมูลที่ต้องการ:**\n    *   ตัวเลข 1 ถึง 10 (จำนวนเต็ม)\n    *   จำนวนตัวเลขที่ต้องการหาผลรวม (จำนวนเต็ม)\n    *   จำนวนตัวเลขที่ต้องการหาผลรวม (จำนวนเต็ม)\n    *   ลักษณะของตัวเลข (เช่น ตัวเลขเป็นจำนวนเต็ม, ตัวเลขเป็นจำนวนเต็มกับตัวเลขอื่น, ตัวเลขเป็นตัวเลขที่ไม่มีตัวประกอบ, ตัวเลขเป็นตัวเลขที่มีตัวประกอบ, ตัวเลขเป็นตัวเลขที่มีตัวประกอบ, ตัวเลขเป็นตัวเลขที่ไม่มีตัวประกอบ, ตัวเลขเป็นตัวเลขที่มีตัวประกอบ, ตัวเลขเป็นตัวเลขที่ไม่มีตัวประกอบ, ตัวเลขเป็นตัวเลขที่มีตัวประกอบ, ตัวเลขเป็นตัวเลขที่ไม่มีตัวประกอบ, ตัวเลขเป็นตัวเลขที่มีตัวประกอบ, ตัวเลขเป็นตัวเลขที่ไม่มีตัวประกอบ, ตัวเลขเป็นตัวเลขที่มีตัวประกอบ, ตัวเลขเป็นตัวเลขที่ไม่มีตัวประกอบ, ตัวเลขเป็นตัวเลขที่มีตัวประกอบ, ตัวเลขเป็นตัวเลขที่ไม่มีตัวประกอบ)\n    *   วิธีการหาผลรวม (เช่น วิธีการหาร, วิธีการหาค่าที่สอง, วิธีการหาค่าที่สาม)\n    *   รูปแบบของผลรวม (เช่น ผลรวมของจำนวนเต็ม, ผล

In [34]:
from IPython.display import display, Markdown

In [35]:
display(Markdown(response))

แน่นอนค่ะ! ผมพร้อมที่จะช่วยคุณเขียนโค้ด Python เพื่อหาผลรวมของตัวเลข 1 ถึง 10

เพื่อให้ผมสามารถช่วยคุณได้ดีที่สุด รบกวนข้อมูลที่คุณมีหน่อยนะคะ

*   **ข้อมูลที่ต้องการ:**
    *   ตัวเลข 1 ถึง 10 (จำนวนเต็ม)
    *   จำนวนตัวเลขที่ต้องการหาผลรวม (จำนวนเต็ม)
    *   จำนวนตัวเลขที่ต้องการหาผลรวม (จำนวนเต็ม)
    *   ลักษณะของตัวเลข (เช่น ตัวเลขเป็นจำนวนเต็ม, ตัวเลขเป็นจำนวนเต็มกับตัวเลขอื่น, ตัวเลขเป็นตัวเลขที่ไม่มีตัวประกอบ, ตัวเลขเป็นตัวเลขที่มีตัวประกอบ, ตัวเลขเป็นตัวเลขที่มีตัวประกอบ, ตัวเลขเป็นตัวเลขที่ไม่มีตัวประกอบ, ตัวเลขเป็นตัวเลขที่มีตัวประกอบ, ตัวเลขเป็นตัวเลขที่ไม่มีตัวประกอบ, ตัวเลขเป็นตัวเลขที่มีตัวประกอบ, ตัวเลขเป็นตัวเลขที่ไม่มีตัวประกอบ, ตัวเลขเป็นตัวเลขที่มีตัวประกอบ, ตัวเลขเป็นตัวเลขที่ไม่มีตัวประกอบ, ตัวเลขเป็นตัวเลขที่มีตัวประกอบ, ตัวเลขเป็นตัวเลขที่ไม่มีตัวประกอบ, ตัวเลขเป็นตัวเลขที่มีตัวประกอบ, ตัวเลขเป็นตัวเลขที่ไม่มีตัวประกอบ)
    *   วิธีการหาผลรวม (เช่น วิธีการหาร, วิธีการหาค่าที่สอง, วิธีการหาค่าที่สาม)
    *   รูปแบบของผลรวม (เช่น ผลรวมของจำนวนเต็ม, ผลรวมของจำนวนเต็มกับตัวเลขอื่น, ผลรวมของตัวเลขที่ไม่มีตัวประกอบ, ผลรวมของตัวเลขที่มีตัวประกอบ, ผลรวมของตัวเลขที่ไม่มีตัวประกอบ, ผลรวมที่ไม่มีตัวประกอบ)
    *   ตัวแปรที่ใช้ในการคำนวณผลรวม (เช่น `sum()`, `product()`, `count()`)
    *   ตัวอย่างโค้ดที่สามารถนำไปใช้ได้

**ตัวอย่างโค้ด (สำหรับข้อมูลที่ต้องการ):**

```python
def find_sum_of_numbers(numbers):
    """
    หาผลรวมของตัวเลข 1 ถึง 10
    """

    if not numbers:
        return 0

    total = 0
    for number in numbers:
        total += number

    return total

# ตัวอย่างการใช้งาน
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
result = find_sum_of_numbers(numbers)
print("ผลรวม:", result)
```

**คำถามเพิ่มเติม:**

*   คุณต้องการให้ผมช่วยคุณเขียนโค้ดเพื่อหาผลรวมของตัวเลข 1 ถึง 10 โดยใช้วิธีการใด?
*   คุณต้องการให้ผมช่วยคุณเขียนโค้ดเพื่อหาผลรวมของตัวเลข 1 ถึง 10 โดยใช้วิธีการใด?
*   คุณต้องการให้ผมช่วยคุณเขียนโค้ดเพื่อหาผลรวมของตัวเลข 1 ถึง 10 โดยใช้รูปแบบของผลรวมอย่างไร?
*   คุณต้องการให้ผมช่วยคุณเขียนโค้ดเพื่อหาผลรวมของตัวเลข 1 ถึง 10 โดยใช้ตัวแปรที่ใช้ในการคำนวณผลรวมอย่างไร?
*   คุณต้องการให้ผมช่วยคุณเขียนโค้ดเพื่อหาผลรวมของตัวเลข 1 ถึง 10 โดยใช้ตัวแปรที่ใช้ในการคำนวณผลรวมอย่างไร?

**คำถามที่ผมไม่สามารถตอบได้:**

*   ผมไม่สามารถหาผลรวมของตัวเลข 1 ถึง 10 ได้โดยตรงจากข้อมูลที่คุณให้มา
*   ผมไม่สามารถหาผลรวมของตัวเลข 1 ถึง 10 โดยใช้วิธีการใด
*   ผมไม่สามารถหาผลรวมของตัวเลข 1 ถึง 10 โดยใช้รูปแบบของผลรวมอย่างไร
*   ผมไม่สามารถหาผลรวมของตัวเลข 1 ถึง 10 โดยใช้ตัวแปรที่ใช้ในการคำนวณผลรวมอย่างไร

**หมายเหตุ:**

*   ผมจะพยายามให้ข้อมูลที่ถูกต้องและครบถ้วนที่สุดเท่าที่ผมมี
*   ผมจะพยายามให้ข้อมูลที่ถูกต้องและเป็นปัจจุบัน
*   ผมจะพยายามให้ข้อมูลที่น่าเชื่อถือและเป็นประโยชน์ต่อคุณ

**โปรดให้ข้อมูลเพิ่มเติมเกี่ยวกับข้อมูลที่คุณต้องการ เช่น:**

*   ข้อมูลจำนวนตัวเลข
*   วิธีการหาผลรวม
*   รูปแบบของผลรวม
*   ตัวแปรที่ใช้ในการคำนวณผลรวม

รอข้อมูลจากคุณอยู่นะครับ!

In [36]:
prompt1 = PromptTemplate(
    template="แปลโจทย์เกี่ยวกับ {question}"
)

chain1 = prompt1 | llm

In [37]:
prompt2 = PromptTemplate(
    template="ทำความเข้าใจโจทย์ที่แปลแล้วต่อไปนี้ {translate}"
)

chain2 = prompt2 | llm

In [38]:
prompt3 = PromptTemplate(
    template="แก้โจทย์ตามที่เข้าใจต่อไปนี้ {understand}"
)

chain3 = prompt3 | llm

In [39]:
prompt4 = PromptTemplate(
    template="อธิบายวิธิีทำจากผลลัทธ์ต่อไปนี้ {solve}"
)

chain4 = prompt4 | llm

In [40]:
parser_chain = StrOutputParser()

In [41]:
full_chain = chain1 | chain2 | chain3 | chain4 | parser_chain

In [42]:
result = full_chain.invoke({"question":"2x + 5 = 15"})

In [43]:
result

'ถูกต้องครับ'

In [44]:
import os
from langchain_openai import ChatOpenAI

In [45]:
OPENAI_API_KEY = "xxx"

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [46]:
llm2 = ChatOpenAI(model="gpt-4o-mini")

In [47]:
chain1 = prompt1 | llm2
chain2 = prompt2 | llm2
chain3 = prompt3 | llm2
chain4 = prompt4 | llm2

full_chain = chain1 | chain2 | chain3 | chain4 | parser_chain

In [48]:
formatted_result = result.replace('\\[', '$$') \
                         .replace('\\]', '$$') \
                         .replace('\\(', '$') \
                         .replace('\\)', '$')

In [49]:
display(Markdown(formatted_result))

ถูกต้องครับ

In [50]:
prompt_template = """
[1. Role/Context บอก LLM ว่าควรทำตัวเป็นอะไร]
คุณเป็น AI ที่เชี่ยวชาญในการวิเคราะห์รีวิวภาษาไทย

[2. Task Description บอกงานที่ต้องทำ]
จาก Review ต่อไปนี้ ให้ Extract 3 keywords
ที่เป็นตัวแทนของ Review นี้ได้ดีที่สุด

[3. Input Data ข้อมูลที่ต้องประมวลผล]
Review: {review}

[4. Output Format บอกรูปแบบของคำตอบ]
ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'keywords'
เช่น {{"keywords": "กาแฟ, บรรยากาศ, ราคา"}

[5. Constraints ข้อจำกัดหรือเงื่อนไข]
- ต้องเป็นคำภาษาไทย
- คั่นด้วย comma
- หากไม่พบให้ตอบ 'ไม่ระบุ'

[6. Examples (optional) ตัวอย่าง input/output]
ตัวอย่าง
Input: "กาแฟอร่อย บรรยากาศดี ราคาแพง"
Output: {"keywords": "กาแฟ, บรรยากาศ, ราคา"}
"""

In [51]:
prompt = """
Extract keywords from this review
{review}
"""

# Pros Simple, fast
# Cons อาจได้ผลไม่ดีกับ Tasks ซับซ้อน

In [52]:
prompt = """
Extract keywords from reviews.

Example 1
Review: "กาแฟอร่อย บริการดี"
Keywords: กาแฟ, บริการ

Example 2
Review: "ราคาแพง แต่คุ้มค่า"
Keywords: ราคา, คุณภาพ

Now extract from this review
Review: {review}
Keywords:
"""

# Pros ได้ผลดีกว่า Zero-shot
# Cons Prompt ใช้ Tokens มากขึ้น

In [53]:
prompt = """
Review: {review}

ขั้นตอนการ Extract keywords
1. อ่านรีวิวและเข้าใจความหมาย
2. ระบุหัวข้อหลักที่กล่าวถึง
3. เลือก 3 keywords ที่สำคัญที่สุด
4. ตอบในรูปแบบ JSON: {{"keywords": "คำ1, คำ2, คำ3"}}

ให้ทำทีละขั้นตอน
"""

# Pros ได้ผลดีกับ Reasoning tasks
# Cons ใช้ Tokens มากกว่า ช้ากว่า

In [54]:
keyword_prompt = PromptTemplate(
template = """
จาก Review ต่อไปนี้
Extract 3 keywords ที่เป็นตัวแทนของ Review นี้

Review: {review}

ขั้นตอนการ Extract keywords
1. อ่านรีวิวและเข้าใจความหมาย
2. ระบุหัวข้อหลักที่กล่าวถึง
3. เลือก 3 keywords ที่สำคัญที่สุด
4. ตอบในรูปแบบ JSON โดยมี key คือ keyword เช่น :{{"keywords": "คำ1, คำ2, คำ3"}}

ให้ทำทีละขั้นตอน
""")

In [55]:
sample_reviews = coffee_reviews_df['column0'].head(2)
sample_reviews

,column0
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...
1,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...


In [56]:
sample_reviews.to_csv('/content/sample_reviews.csv', index=False)

In [57]:
keyword_extract = keyword_prompt | llm | StrOutputParser()

In [58]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    cleaned = (text).replace("**", "").strip()
    results.append({"review": review, "text": cleaned})

CPU times: user 69.7 ms, sys: 8.76 ms, total: 78.5 ms
Wall time: 7.37 s


In [59]:
for result in results:
    print(f"Review {result['review'][:100]}...")
    print(f"text {result['text']}\n")

Review ร้านอาหารใหญ่มากกกกกกก 
เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี 
ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ...
text ```json
{
  "keywords": [
    "ร้านอาหารใหญ่",
    "ห้องน้ำ",
    "กาแฟ",
    "น้ำผึ้ง",
    "น้ำผึ้งมาราด",
    "แพงเวอร์",
    "อย่าสั่งเลย",
    "ลาบไข่ต้ม",
    "ไข่มันคาว",
    "ไม่ประทับใจ",
    "ทอดมันหัวปลีกรอบอร่อย",
    "เบิ้ล"
  ]
}
```

Review วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์

ก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก ...
text ```json
{
  "keywords": [
    "ลาเต้",
    "ร้อน",
    "กาแฟ",
    "ร้านวาวี",
    "อารีย์",
    "Starbuck",
    "แฟนๆ Starbuck",
    "กาแฟ",
    "ประสบการณ์",
    "ราคา",
    "อาหาร",
    "การกิน",
    "การชิม",
    "อร่อย",
    "ดี",
    "ราคาไม่แพง",
    "กินบ่อย",
    "จุกได้",
    "บ่อยก็",
    "จุกๆ",
    "อาจจะ",
    "อาจจะ",
    "ได้",
    "ไม่ค่อย",
    "เข้าใจ",
    "รสชาติ",
    "กลิ่น",
    "มวยรุ่นเดียวกะ Starbuck",
    "เมพพพพ",
    "ไม่ค่อย",
    "ไม่รู้เรื่อง",
    "กาแฟมาก",
    "กาแฟดี",
    "กาแ

In [60]:
keyword_extract = keyword_prompt | llm2 | StrOutputParser()

In [61]:
for result in results:
    print(f"Review {result['review'][:100]}...")
    print(f"text {result['text']}\n")

Review ร้านอาหารใหญ่มากกกกกกก 
เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี 
ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ...
text ```json
{
  "keywords": [
    "ร้านอาหารใหญ่",
    "ห้องน้ำ",
    "กาแฟ",
    "น้ำผึ้ง",
    "น้ำผึ้งมาราด",
    "แพงเวอร์",
    "อย่าสั่งเลย",
    "ลาบไข่ต้ม",
    "ไข่มันคาว",
    "ไม่ประทับใจ",
    "ทอดมันหัวปลีกรอบอร่อย",
    "เบิ้ล"
  ]
}
```

Review วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์

ก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก ...
text ```json
{
  "keywords": [
    "ลาเต้",
    "ร้อน",
    "กาแฟ",
    "ร้านวาวี",
    "อารีย์",
    "Starbuck",
    "แฟนๆ Starbuck",
    "กาแฟ",
    "ประสบการณ์",
    "ราคา",
    "อาหาร",
    "การกิน",
    "การชิม",
    "อร่อย",
    "ดี",
    "ราคาไม่แพง",
    "กินบ่อย",
    "จุกได้",
    "บ่อยก็",
    "จุกๆ",
    "อาจจะ",
    "อาจจะ",
    "ได้",
    "ไม่ค่อย",
    "เข้าใจ",
    "รสชาติ",
    "กลิ่น",
    "มวยรุ่นเดียวกะ Starbuck",
    "เมพพพพ",
    "ไม่ค่อย",
    "ไม่รู้เรื่อง",
    "กาแฟมาก",
    "กาแฟดี",
    "กาแ

In [62]:
!ollama pull scb10x/typhoon2.1-gemma3-4b

In [63]:
!ollama list

NAME                                  ID              SIZE      MODIFIED               
scb10x/typhoon2.1-gemma3-4b:latest    8cfab9097c9d    2.6 GB    Less than a second ago    
gemma3:270m                           e7d36fb2c3b3    291 MB    About a minute ago        


In [64]:
llm3 = OllamaLLM(model="gemma3:270m")

In [65]:
keyword_extract = keyword_prompt | llm3 | StrOutputParser()

In [66]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    cleaned = (text).replace("**", "").strip()
    results.append({"review": review, "text": cleaned})

CPU times: user 5.58 s, sys: 526 ms, total: 6.1 s
Wall time: 4min 12s


In [67]:
for result in results:
    print(f"Review {result['review'][:100]}...")
    print(f"text {result['text']}\n")

Review ร้านอาหารใหญ่มากกกกกกก 
เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี 
ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ...
text ```json
{
  "keywords": [
    "ร้านอาหารใหญ่",
    "ห้องน้ำ",
    "กาแฟ",
    "น้ำผึ้ง",
    "น้ำผึ้งมาราด",
    "แพงเวอร์",
    "ราคูกำลังเลิก",
    "ไม่ประทับใจ",
    "ขนมหวาน",
    "ฟรีเล็กน้อย"
  ]
}
```

Review วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์

ก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก ...
text ```json
{
  "keywords": [
    "ลาเต้",
    "ร้อนแก้วนึง",
    "ซดกาแฟ",
    "กาแฟ",
    "Starbuck",
    "การกินบ่อย",
    "กาแฟร้านวาวี",
    "ร้านวาวี",
    "อารีย์",
    "ร้าน Software Park",
    "ไรกินแถวนี้",
    "ร้านอาหาร",
    "ทานกาแฟ",
    "กาแฟทาน",
    "กาแฟร้านวาวี",
    "กาแฟร้าน Software Park",
    "กาแฟร้านวาวี",
    "กาแฟของเพื่อน",
    "กาแฟของร้านวาวี",
    "กาแฟร้าน Software Park",
    "กาแฟราคาไม่แพง",
    "กาแฟกินบ่อย",
    "กาแฟจุกได้",
    "กาแฟน่าทาน",
    "กาแฟอร่อย",
    "กาแฟรสชาติไม่ธรรมดา",
    "กาแฟไม่ธรรมดา",
    "กาแฟมวยร

In [68]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

class ReviewKeywords(BaseModel):
    keywords: str = Field(
        ...,  # Required field
        description="3 keywords separated by comma",
        examples=["กาแฟ, บรรยากาศ, ราคา"]
    )

class KeywordsExtraction(BaseModel):
    extracted_qualities: ReviewKeywords

In [69]:
keywords_parser = JsonOutputParser(pydantic_object=KeywordsExtraction)

In [70]:
keyword_extract = keyword_prompt | llm3 | keywords_parser

In [71]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    results.append({"review": review, "text": text})

CPU times: user 56.3 ms, sys: 5.89 ms, total: 62.2 ms
Wall time: 7.2 s


In [72]:
results[0]['text']['keywords']

['ร้านอาหารใหญ่',
 'ห้องน้ำ',
 'กาแฟ',
 'น้ำผึ้ง',
 'ลาบไข่ต้ม',
 'ไข่มันคาว',
 'ขนมไทย',
 'ขนมหวาน']

In [73]:
results[1]['text']['keywords']

['กาแฟ',
 'ร้านวาวี',
 'อารีย์',
 'Starbuck',
 'แฟนๆ Starbuck',
 'กาแฟที่นี่',
 'อาหาร',
 'อาหารกลางวัน',
 'อาหารเย็น',
 'ร้านอาหาร',
 'วิว',
 'สวนบ้าน',
 'ต้นไม้',
 'สีเขียว',
 'สีน้ำตาล',
 'เมพพพ',
 'รสชาติ',
 'กลิ่น',
 'มวยรุ่นเดียวกะ Starbuck',
 'หน้าตา',
 'รสชาติ',
 'เมนูอื่นๆ',
 'กาแฟ',
 'อาหาร',
 'ราคา',
 'กินบ่อย',
 'จุกได้',
 'ซากอารยธรรมลาเต้เมพ']

In [74]:
llm3 = OllamaLLM(
    model="scb10x/typhoon2.1-gemma3-4b",
    temperature=0.05
)

keyword_extract = keyword_prompt | llm3 | keywords_parser

In [75]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    results.append({"review": review, "text": text})

CPU times: user 154 ms, sys: 13.6 ms, total: 168 ms
Wall time: 17.8 s


In [76]:
results[0]['text']['keywords'], results[1]['text']['keywords']

(['ราคา', 'รสชาติ', 'ขนาด'], ['กาแฟ', 'บรรยากาศ', 'วาวี'])

In [77]:
keyword_prompt = PromptTemplate(
    template="""จาก Review ต่อไปนี้
    Extract keywords 3 keywords เท่านั้น ที่เป็นตัวแทนของ Review นี้ได้ดีที่สุด

    Review: {review}

    ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'keywords'
    และค่าเป็นข้อความที่ดึงได้คั่นด้วย comma
    เช่น {{"keywords": "กาแฟ, บรรยากาศ, ราคา"}}

    หากไม่พบข้อมูลให้ตอบ 'ไม่ระบุ'
    """
)

In [78]:
keyword_extract = keyword_prompt | llm3 | keywords_parser

In [79]:
inputs = [{"review": r} for r in sample_reviews]

In [80]:
%%time
results = keyword_extract.batch(inputs)

CPU times: user 19.6 ms, sys: 2.29 ms, total: 21.9 ms
Wall time: 2.34 s


In [81]:
results

[{'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'},
 {'keywords': 'กาแฟ, บรรยากาศ, รสชาติ'}]

In [82]:
sample_reviews_1000 = coffee_reviews_df['column0'].head(1000).tolist()
inputs_1000 = [{"review": r} for r in sample_reviews_1000]

BATCH_SIZE = 8

results_1000 = []

In [83]:
from tqdm import tqdm

In [84]:
for i in tqdm(range(0, len(inputs_1000), BATCH_SIZE), desc="Extracting keywords"):
    batch = inputs_1000[i:i+BATCH_SIZE]
    outputs = keyword_extract.batch(batch)
    for inp, out in zip(batch, outputs):
        results_1000.append({"review": inp["review"], "text": out})

Extracting keywords: 100%|██████████| 125/125 [19:50<00:00,  9.53s/it]


In [85]:
for result in results_1000[0:10]:
    print(result['text']['keywords'])

ราคา, บรรยากาศ, ไข่ต้ม
กาแฟ, บรรยากาศ, รสชาติ
โดนัท, รสชาติ, ความหวาน
วาซาบิ, ปลาซาบะ, ข้าวหน้าปลา
โดนัท, Original Glazed, คิว
วังพญาไท, นรสิงห์, คาเฟ่
อาหารทะเล, ราคา, บรรยากาศ
กาแฟ, เค้ก, พนักงาน
ฮันนี่ โทสท์, ราคา, รสชาติ
คาเฟ่, วานิลลา, คาโบนาร่า


In [86]:
  results_1000[0]

{'review': 'ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี \nห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย \nลาบไข่ต้ม ไข่มันคาวอะ เลยไม่ประทับใจเท่าไหร่\nทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล \nพะแนงห่อไข่อร่อยดี เห้ยแต่ราคา 150บาทมันเกินไปนะ รับไม่ไหวว\nเลิกกินแล้วมีขนมหวานให้กินฟรีเล็กน้อย )ขนมไทย) \n\nคงไม่ไปซ้ำ แพงเกิน ',
 'text': {'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'}}

In [87]:
context_prompt = PromptTemplate(
    template="""จาก Review ต่อไปนี้
    Extract context ใน Review ที่เป็นตัวแทนที่บรรยายคำว่า {keyword} ได้ดีที่สุด

    Review: {review}

    ตอบผลลัพธ์ในรูปแบบ JSON โดยมีคีย์เป็น 'context'
    และค่าเป็นข้อความที่ดึงได้จาก Review

    ตัวอย่าง
    keyword คือ "กาแฟ"
    context: "กาแฟที่นี่อร่อยมาก หอมกลิ่นกาแฟคั่วสด"

    หากไม่พบข้อมูลให้ตอบ 'ไม่ระบุ'
    """
)

In [88]:
sentiment_prompt = PromptTemplate(
    template="""จากข้อความต่อไปนี้ ให้วิเคราะห์ว่าเป็น sentiment แบบไหน
    เลือกจากตัวเลือกเหล่านี้เท่านั้น: "positive", "negative", "neutral"

    ข้อความ: {message}

    ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'sentiment'
    และค่าเป็นหนึ่งใน list ต่อไปนี้ [positive, negative, neutral]

    คำแนะนำ
    - positive คือ ข้อความที่แสดงความพอใจ ชอบ ดี
    - negative คือ ข้อความที่แสดงความไม่พอใจ ไม่ชอบ แย่
    - neutral คือ ข้อความที่ไม่แสดงความรู้สึกชัดเจน เป็นกลาง

    # ตัวอย่าง
    # ข้อความ "อร่อยมาก" → {{"sentiment": "positive"}}
    # ข้อความ "แพงไป" → {{"sentiment": "negative"}}
    # ข้อความ "ปกติ" → {{"sentiment": "neutral"}}
    """
)


In [89]:
class ContextKeyword(BaseModel):
    context: str = Field(..., description="context that best represent this keyword")

class ContexExtraction(BaseModel):
    extracted: ContextKeyword

context_parser = JsonOutputParser(pydantic_object=ContexExtraction)

In [90]:
context_extract = context_prompt | llm3 | context_parser

In [91]:
class Sentiment(BaseModel):
    sentiment: str = Field(..., description="sentiment of review")

class SentimentExtraction(BaseModel):
    extracted: Sentiment


sentiment_parser = JsonOutputParser(pydantic_object=SentimentExtraction)

In [92]:
sentiment_extract = sentiment_prompt | llm3 | sentiment_parser

In [93]:
keywords = results_1000[0]['text']['keywords']


In [94]:
context_inputs = []

for review_data in tqdm(results_1000, desc="Preparing context inputs"):
    review = review_data['review']
    keywords = review_data['text']['keywords']
    for keyword in keywords:
        context_inputs.append({"review": review, "keyword": keyword.strip()})

Preparing context inputs: 100%|██████████| 1000/1000 [00:00<00:00, 100698.74it/s]


In [95]:
len(context_inputs)

24770

In [96]:
context_results = []
for i in tqdm(range(len(results_1000)), desc="Extracting contexts"):
   result = context_extract.batch(context_inputs[i*3:(i*3)+3])
   contexts_list = [d['context'] for d in result]
   results_1000[i]['contexts'] = contexts_list

Extracting contexts: 100%|██████████| 1000/1000 [1:26:18<00:00,  5.18s/it]


In [97]:
results_1000[:2]

[{'review': 'ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี \nห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย \nลาบไข่ต้ม ไข่มันคาวอะ เลยไม่ประทับใจเท่าไหร่\nทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล \nพะแนงห่อไข่อร่อยดี เห้ยแต่ราคา 150บาทมันเกินไปนะ รับไม่ไหวว\nเลิกกินแล้วมีขนมหวานให้กินฟรีเล็กน้อย )ขนมไทย) \n\nคงไม่ไปซ้ำ แพงเกิน ',
  'text': {'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'},
  'contexts': ['ร้านอาหารใหญ่มากกกกกกก',
   'ร้านอาหารใหญ่มากกกกกกก เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี',
   'ห้องทานอาหารอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย']},
 {'review': 'วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์\n\nก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก Twitter ว่ากาแฟที่นี่อร่อยมากกกกกกก\nเรียกว่าระดับแฟนๆ Starbuck ยังต้องเหลียวมามอง ก็ดองมานานจนถึงวันนี้โอกาสเหมาะ ไป\npresent งานที่ตึก Software Park เสร็จก็เลยมาหาไรกินแถวนี้พอดี\nจัดไป.....\n\n\nบรรยากาศรอบๆร้านก็แต่งได้น่านั่ง อารมณ์ประมาณว่าอยู่ในสว

In [98]:
sentiment_inputs = []

for review_data in tqdm(results_1000, desc="Preparing sentiment inputs"):
    contexts = review_data['contexts']
    for context in contexts:
        sentiment_inputs.append({"message": context})

Preparing sentiment inputs: 100%|██████████| 1000/1000 [00:00<00:00, 290786.47it/s]


In [99]:
len(sentiment_inputs)

3000

In [100]:
sentiment_inputs[:6]

[{'message': 'ร้านอาหารใหญ่มากกกกกกก'},
 {'message': 'ร้านอาหารใหญ่มากกกกกกก เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี'},
 {'message': 'ห้องทานอาหารอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย'},
 {'message': 'ร้านอาหารใหญ่มากกกกกกก เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี'},
 {'message': 'ราคา 150 บาทมันเกินไปนะ รับไม่ไหวว'},
 {'message': 'ทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล'}]

In [101]:
for i in tqdm(range(len(results_1000)), desc="Extracting sentiment"):
   result = sentiment_extract.batch(sentiment_inputs[i*3:(i*3)+3])
   sentiment_list = [d['sentiment'] for d in result]
   results_1000[i]['sentiment'] = sentiment_list

Extracting sentiment: 100%|██████████| 1000/1000 [42:33<00:00,  2.55s/it]


In [102]:
results_1000[0:2]

[{'review': 'ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี \nห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย \nลาบไข่ต้ม ไข่มันคาวอะ เลยไม่ประทับใจเท่าไหร่\nทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล \nพะแนงห่อไข่อร่อยดี เห้ยแต่ราคา 150บาทมันเกินไปนะ รับไม่ไหวว\nเลิกกินแล้วมีขนมหวานให้กินฟรีเล็กน้อย )ขนมไทย) \n\nคงไม่ไปซ้ำ แพงเกิน ',
  'text': {'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'},
  'contexts': ['ร้านอาหารใหญ่มากกกกกกก',
   'ร้านอาหารใหญ่มากกกกกกก เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี',
   'ห้องทานอาหารอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย'],
  'sentiment': ['neutral', 'neutral', 'negative']},
 {'review': 'วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์\n\nก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก Twitter ว่ากาแฟที่นี่อร่อยมากกกกกกก\nเรียกว่าระดับแฟนๆ Starbuck ยังต้องเหลียวมามอง ก็ดองมานานจนถึงวันนี้โอกาสเหมาะ ไป\npresent งานที่ตึก Software Park เสร็จก็เลยมาหาไรกินแถวนี้พอดี\nจัดไป.....\n\n\nบรรยา

In [103]:
import pandas as pd

df = pd.DataFrame([
    {
        'review': r['review'],
        **{f'keyword{i+1}': k.strip() for i, k in enumerate(r['text']['keywords'])},
        **{f'context{i+1}_keyword': c for i, c in enumerate(r['contexts'])},
        **{f'sentiment{i+1}': s for i, s in enumerate(r['sentiment'])}
    }
    for r in results_1000
])

In [104]:
df.to_csv('results.csv', index=False, encoding='utf-8-sig')